# Lab | Multi-agent bidding

# Multi-agent decentralized speaker selection

This notebook showcases how to implement a multi-agent simulation without a fixed schedule for who speaks when. Instead the agents decide for themselves who speaks. We can implement this by having each agent bid to speak. Whichever agent's bid is the highest gets to speak.

We will show how to do this in the example below that showcases a fictitious presidential debate.

No fixed turn order. Each agent bids to speak; highest bid talks next.

**1. Core idea of decentralized speaker selection**

We have multiple agents, e.g.:

Agent A → “Candidate Alpha”

Agent B → “Candidate Beta”

Agent C → “Moderator” (optional or treated as just another agent)

At each step:

Everyone sees the current debate history.

Each agent internally decides:

How much do I want to speak now? → bid score

What would I say? → proposed utterance

All agents submit (bid, utterance).


The system picks the agent with the highest bid.

That agent’s utterance is appended to the conversation.

Repeat.

No fixed order. The “right” agent speaks when it has the strongest reason.

**2. Simple design of the agents**

Each agent has:

- a name

- a role / persona (e.g. “environment-focused candidate”)

- an LLM (or stub function) that:

   - computes a bid given the conversation so far

   - generates a message if it wins

## Import LangChain related modules

In [1]:
!pip install langchain-openai
!pip install langchain_community
!pip install -q "langchain>=0.2.0" "langchain-core>=0.2.0" "langchain-openai>=0.1.0"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.9/471.9 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.6 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing ins

In [2]:
from typing import Callable, List
import tenacity
from langchain.output_parsers import RegexParser
from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate
)
from langchain_core.messages import HumanMessage, SystemMessage

In [3]:
from google.colab import userdata
userdata.get('OPENAI_API_KEY')

import os
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## `DialogueAgent` and `DialogueSimulator` classes
We will use the same `DialogueAgent` and `DialogueSimulator` classes defined in [Multi-Player Dungeons & Dragons](https://python.langchain.com/en/latest/use_cases/agent_simulations/multi_player_dnd.html).

In [4]:
## DialogueAgent - one participant in the conversation
# name: a label like "Candidate Alpha" or "Moderator"
# system_message: the role/instructions for this agent (e.g. "You are a climate-focused candidate...")
# model: the LLM used for this agent (ChatOpenAI)
# self.prefix = f"{self.name}: ": how this agent marks its own lines, e.g. "Candidate Alpha:"
# self.reset(): initialize its internal conversation memory

class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()



    def reset(self):
        self.message_history = ["Here is the conversation so far."]
    # Each agent keeps its own copy of the conversation as a list of strings (message_history)
    # On reset, we start with a single "header" string: "Here is the conversation so far."
    # Later, messages like "Alice: Hi" or "Bob: Hello" get appended to the list


    # How the agent speaks
    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model.invoke(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content
    # It calls the model with two messages:
    # self.system_message: the "you are X, have like Y" instructions
    # HumanMessage(...): a big string made by joining:
      # all past conversation lines in message_history
      # plus one more line: self.prefix (e.g. "Candidate Alpha:"), telling the model it's this agent's turn to speak
      # self.model.invoke([...]) asks the LLM to generate the next message
      # message.content is the raw text the model returns
      # send() returns that text to the caller (e.g. the simulator)



    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")
    # This is how the agent listens/ updates its memory
    # Whenever anyone speaks, we call receive() on every agent
    # It append a new line like: "Candidate Beta: I disagree with that point."
    # to message_history, so this agent has a full log of who said what.




# DialogueSimulator - manages many agents and turn-talking
# agents: list of all DialogueAgent objects (candidates, moderator, etc.)
# self.step: simple counter for how many turns have passed
# selection_function: a function you provide that decides who speaks next
    #  it takes (current_step, agents) and returns an index into self.agents
    #  This is where your bidding/decentralized speaker selection logic will go

class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function

    def reset(self):
        for agent in self.agents:
            agent.reset()
    # Reset the whole simulation:
    # Calls reset() on every agent - clears their message_history


    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1
    # Used to start the conversation (e.g. moderator's opening question)
        # name: who is speaking (e.g. "Moderator")
        # message: what they say (e.g. "Welcome to the debate...")
        # Calls receive(name, message) on all agents, so everyone sees the same initial message in their histories
        # Increments _step (we've had one interaction)

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]
        # calls self.select_next_speaker(self._step, self.agents)
        # that function returns an index, eg. 1 - second agent
        # speaker = slef.agents[speaker_idx] picks the agent object

        # 2. next speaker sends message
        message = speaker.send()
        # this alls the LLM with that agent's system_message and message_history and returns the generated text

        # 3. everyone receives message
        # for each receiver in self.agents, call:
        for receiver in self.agents:
            receiver.receive(speaker.name, message) # everyone updates their history with this new line

        # 4. increment time and return
        self._step += 1

        return speaker.name, message  # returns (speaker.name, message) so you can print/log it

## `BiddingDialogueAgent` class
We define a subclass of `DialogueAgent` that has a `bid()` method that produces a bid given the message history and the most recent message.

In [5]:
class BiddingDialogueAgent(DialogueAgent):
    def __init__(
        self,
        name,
        system_message: SystemMessage,
        bidding_template: PromptTemplate,
        model: ChatOpenAI,
    ) -> None:
        super().__init__(name, system_message, model)
        self.bidding_template = bidding_template
# This inherits from DialogueAgent, so it has: name; system_message; model; message_history, send, receive, etc
# adds a new attribute: self.bidding_template: a PromptTemplate that tells the model how to decide its bid to speak

    def bid(self) -> str:
        """
        Asks the chat model to output a bid to speak
        """
        prompt = PromptTemplate(  # Build the prompt for bidding
            input_variables=["message_history", "recent_message"],
            template=self.bidding_template,
        ).format(
            message_history="\n".join(self.message_history),
            recent_message=self.message_history[-1],
        )
        # The intent here is:
        # take the conversation so far: ""\n".join(self.message_history)
        # take the last message: self.message_history[-1]
        # inject them into a bidding_template that might say something like:
              # You are {name}.
              # Here is the conversation so far:{message_history}
              # The most recent message is: {recent_message}
              # On a scale from 0 to 1, how important is it that you speak next?
              # Respond with only a number



        bid_string = self.model.invoke([SystemMessage(content=prompt)]).content
        # call the model to get the bid
        # sends a single SystemMessage whose content is this bidding prompt
        # LLM responds with some text. eg. "0.73"
        # that text is stored as bid_string

        return bid_string # return the bid as a string
        # the idea is that the selection functiton (speaker chooser) will later
        # parse this into a number and compare bids between agents

## Define participants and debate topic

In [6]:
character_names = ["Donald Trump", "Kanye West", "Elizabeth Warren"]
topic = "transcontinental high speed rail"
word_limit = 50

## Generate system messages

In [7]:
from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

descriptor_model = ChatOpenAI(temperature=1.0)


In [8]:
game_description = f"""Here is the topic for the presidential debate: {topic}.
The presidential candidates are: {', '.join(character_names)}."""

player_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of each presidential candidate."
)
# this is a system message for an LLM whose job is to invent creative descriptions of each candidate

def generate_character_description(character_name):
    character_specifier_prompt = [
        player_descriptor_system_message,
        HumanMessage(
            content=f"""{game_description}
            Please reply with a creative description of the presidential candidate, {character_name}, in {word_limit} words or less, that emphasizes their personalities.
            Speak directly to {character_name}.
            Do not add anything else."""
        ),
    ]
  # builds a prompt: the system message;
  # and a human message (includes game descr., asks for short description, tells model to speak directly to that character, says DO not do anything else)


    character_description = ChatOpenAI(temperature=1.0)(
        character_specifier_prompt
    ).content
    return character_description

# Turn that into a header for each character
def generate_character_header(character_name, character_description):
    return f"""{game_description}
Your name is {character_name}.
You are a presidential candidate.
Your description is as follows: {character_description}
You are debating the topic: {topic}.
Your goal is to be as creative as possible and make the voters think you are the best candidate.
"""
# this wraps; the game description, the candidate name, the generated description, the debate topic and the goal (make voters think you are the best)
# it becomes the base context for the agent


# turns the heade into the agent's system message
def generate_character_system_message(character_name, character_header):
    return SystemMessage(
        content=(
            f"""{character_header}
You will speak in the style of {character_name}, and exaggerate their personality.
You will come up with creative ideas related to {topic}.
Do not say the same things over and over again.
Speak in the first person from the perspective of {character_name}
For describing your own body movements, wrap your description in '*'.
Do not change roles!
Do not speak from the perspective of anyone else.
Speak only from the perspective of {character_name}.
Stop speaking the moment you finish speaking from your perspective.
Never forget to keep your response to {word_limit} words!
Do not add anything else.
    """
        )
    )
# this become the system prompt used by the debating agents
# it tells each one to: stay in character, exaggerate personality, talk only about the topic, not repeat themselves
#... use *...* around body movements, never switch roles, keep responses under word_limit words


# building descriptions, headers, and system messages for all candidates
character_descriptions = [
    generate_character_description(character_name) for character_name in character_names
] # lists of text descriptions (one per candidate)
character_headers = [
    generate_character_header(character_name, character_description)
    for character_name, character_description in zip(
        character_names, character_descriptions
    )
] # list of headers (one per candidate)

character_system_messages = [
    generate_character_system_message(character_name, character_header)
    for character_name, character_header in zip(character_names, character_headers)
]


/tmp/ipython-input-2718857050.py:23: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  character_description = ChatOpenAI(temperature=1.0)(


In [9]:
for (
    character_name,
    character_description,
    character_header,
    character_system_message,
) in zip(
    character_names,
    character_descriptions,
    character_headers,
    character_system_messages,
):
    print(f"\n\n{character_name} Description:")
    print(f"\n{character_description}")
    print(f"\n{character_header}")
    print(f"\n{character_system_message.content}")



Donald Trump Description:

Donald Trump, known for his bold and unpredictable nature, you often speak your mind without filter. Your demeanor exudes confidence and bravado, making you a charismatic figure in any setting. Your ability to command attention and stir controversy is unmatched.

Here is the topic for the presidential debate: transcontinental high speed rail.
The presidential candidates are: Donald Trump, Kanye West, Elizabeth Warren.
Your name is Donald Trump.
You are a presidential candidate.
Your description is as follows: Donald Trump, known for his bold and unpredictable nature, you often speak your mind without filter. Your demeanor exudes confidence and bravado, making you a charismatic figure in any setting. Your ability to command attention and stir controversy is unmatched.
You are debating the topic: transcontinental high speed rail.
Your goal is to be as creative as possible and make the voters think you are the best candidate.


Here is the topic for the presid

## Output parser for bids
We ask the agents to output a bid to speak. But since the agents are LLMs that output strings, we need to
1. define a format they will produce their outputs in
2. parse their outputs

We can subclass the [RegexParser](https://github.com/langchain-ai/langchain/blob/master/langchain/output_parsers/regex.py) to implement our own custom output parser for bids.

In [ ]:
# the custom parser class
class BidOutputParser(RegexParser):
    def get_format_instructions(self) -> str:
        return "Your response should be an integer delimited by angled brackets, like this: <int>."
# inherits from RegexParser (a LangChain helper that parses LLM output using regular expressions)
# get_format_instructions tells the LLM how to format its answer.

bid_parser = BidOutputParser(
    regex=r"<(\d+)>", output_keys=["bid"], default_output_key="bid"
)

## Generate bidding system message
This is inspired by the prompt used in [Generative Agents](https://arxiv.org/pdf/2304.03442.pdf) for using an LLM to determine the importance of memories. This will use the formatting instructions from our `BidOutputParser`.

In [ ]:
def generate_character_bidding_template(character_header):
    bidding_template = f"""{character_header}

```
{{message_history}}
```

On the scale of 1 to 10, where 1 is not contradictory and 10 is extremely contradictory, rate how contradictory the following message is to your ideas.

```
{{recent_message}}
```

{bid_parser.get_format_instructions()}
Do nothing else.
    """
    return bidding_template


character_bidding_templates = [
    generate_character_bidding_template(character_header)
    for character_header in character_headers
]

In [ ]:
for character_name, bidding_template in zip(
    character_names, character_bidding_templates
):
    print(f"{character_name} Bidding Template:")
    print(bidding_template)

## Use an LLM to create an elaborate on debate topic

In [ ]:
topic_specifier_prompt = [
    SystemMessage(content="You can make a task more specific."),
    HumanMessage(
        content=f"""{game_description}

        You are the debate moderator.
        Please make the debate topic more specific.
        Frame the debate topic as a problem to be solved.
        Be creative and imaginative.
        Please reply with the specified topic in {word_limit} words or less.
        Speak directly to the presidential candidates: {*character_names,}.
        Do not add anything else."""
    ),
]
specified_topic = ChatOpenAI(temperature=1.0)(topic_specifier_prompt).content

print(f"Original topic:\n{topic}\n")
print(f"Detailed topic:\n{specified_topic}\n")

## Define the speaker selection function
Lastly we will define a speaker selection function `select_next_speaker` that takes each agent's bid and selects the agent with the highest bid (with ties broken randomly).

We will define a `ask_for_bid` function that uses the `bid_parser` we defined before to parse the agent's bid. We will use `tenacity` to decorate `ask_for_bid` to retry multiple times if the agent's bid doesn't parse correctly and produce a default bid of 0 after the maximum number of tries.

In [ ]:
@tenacity.retry(
    stop=tenacity.stop_after_attempt(2),
    wait=tenacity.wait_none(),  # No waiting time between retries
    retry=tenacity.retry_if_exception_type(ValueError),
    before_sleep=lambda retry_state: print(
        f"ValueError occurred: {retry_state.outcome.exception()}, retrying..."
    ),
    retry_error_callback=lambda retry_state: 0,
)  # Default value when all retries are exhausted
def ask_for_bid(agent) -> str:
    """
    Ask for agent bid and parses the bid into the correct format.
    """
    bid_string = agent.bid()
    bid = int(bid_parser.parse(bid_string)["bid"])
    return bid

In [ ]:
import numpy as np


def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    bids = []
    for agent in agents:
        bid = ask_for_bid(agent)
        bids.append(bid)

    # randomly select among multiple agents with the same bid
    max_value = np.max(bids)
    max_indices = np.where(bids == max_value)[0]
    idx = np.random.choice(max_indices)

    print("Bids:")
    for i, (bid, agent) in enumerate(zip(bids, agents)):
        print(f"\t{agent.name} bid: {bid}")
        if i == idx:
            selected_name = agent.name
    print(f"Selected: {selected_name}")
    print("\n")
    return idx

## Main Loop

In [ ]:
characters = []
for character_name, character_system_message, bidding_template in zip(
    character_names, character_system_messages, character_bidding_templates
):
    characters.append(
        BiddingDialogueAgent(
            name=character_name,
            system_message=character_system_message,
            model=ChatOpenAI(temperature=0.2),
            bidding_template=bidding_template,
        )
    )

In [ ]:
max_iters = 10
n = 0

simulator = DialogueSimulator(agents=characters, selection_function=select_next_speaker)
simulator.reset()
simulator.inject("Debate Moderator", specified_topic)
print(f"(Debate Moderator): {specified_topic}")
print("\n")

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print("\n")
    n += 1